In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
maxDt <- (indices$NseConstituents() %>%
          filter(NAME == "NIFTY 50") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$NseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "NIFTY 50") %>%
     print(n = 10)

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:   lazy query [?? x 5]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
   NAME     TIME_STAMP SYMBOL     INDUSTRY           CAP_WEIGHT
   <chr>    <date>     <chr>      <chr>                   <dbl>
 1 NIFTY 50 2019-08-16 ADANIPORTS SERVICES                0.665
 2 NIFTY 50 2019-08-16 ASIANPAINT CONSUMER GOODS          1.47 
 3 NIFTY 50 2019-08-16 AXISBANK   FINANCIAL SERVICES      3.15 
 4 NIFTY 50 2019-08-16 BAJAJ-AUTO AUTOMOBILE              0.745
 5 NIFTY 50 2019-08-16 BAJAJFINSV FINANCIAL SERVICES      0.969
 6 NIFTY 50 2019-08-16 BAJFINANCE FINANCIAL SERVICES      1.72 
 7 NIFTY 50 2019-08-16 BHARTIARTL TELECOM                 1.45 
 8 NIFTY 50 2019-08-16 BPCL       ENERGY                  0.610
 9 NIFTY 50 2019-08-16 BRITANNIA  CONSUMER GOODS          0.683
10 NIFTY 50 2019-08-16 CIPLA      PHARMA                  0.579
# … with more rows


In [3]:
maxDt <- (indices$NseConstituents() %>%
          filter(NAME == "NIFTY 50") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$NseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "NIFTY 50") %>%
     print(n = 10)

# Source:   lazy query [?? x 5]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
   NAME     TIME_STAMP SYMBOL     INDUSTRY           CAP_WEIGHT
   <chr>    <date>     <chr>      <chr>                   <dbl>
 1 NIFTY 50 2019-08-16 ADANIPORTS SERVICES                0.665
 2 NIFTY 50 2019-08-16 ASIANPAINT CONSUMER GOODS          1.47 
 3 NIFTY 50 2019-08-16 AXISBANK   FINANCIAL SERVICES      3.15 
 4 NIFTY 50 2019-08-16 BAJAJ-AUTO AUTOMOBILE              0.745
 5 NIFTY 50 2019-08-16 BAJAJFINSV FINANCIAL SERVICES      0.969
 6 NIFTY 50 2019-08-16 BAJFINANCE FINANCIAL SERVICES      1.72 
 7 NIFTY 50 2019-08-16 BHARTIARTL TELECOM                 1.45 
 8 NIFTY 50 2019-08-16 BPCL       ENERGY                  0.610
 9 NIFTY 50 2019-08-16 BRITANNIA  CONSUMER GOODS          0.683
10 NIFTY 50 2019-08-16 CIPLA      PHARMA                  0.579
# … with more rows


In [4]:
maxDtBse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtBse) %>%
     select(NAME) %>%
     print(n=10)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME                     
   <chr>                    
 1 NIFTY 100                
 2 NIFTY 100 TR             
 3 Nifty 1D Rate Index      
 4 NIFTY 200                
 5 NIFTY 200 TR             
 6 NIFTY 50                 
 7 NIFTY 50 ARBITRAGE       
 8 NIFTY 50 FUTURES INDEX   
 9 NIFTY 50 FUTURES TR INDEX
10 NIFTY 50 TR              
# … with more rows


In [5]:
class(maxDtBse)

[1] "Date"

In [6]:
class(indices)

[1] "Indices"
attr(,"package")
[1] "plutoR"

In [7]:
maxDtBse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtBse) %>%
     select(NAME) %>%
     print(n=5)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Groups:   NAME
  NAME               
  <chr>              
1 NIFTY 100          
2 NIFTY 100 TR       
3 Nifty 1D Rate Index
4 NIFTY 200          
5 NIFTY 200 TR       
# … with more rows


In [8]:
maxDtBse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtBse) %>%
     select(NAME) %>%
     print(n=50)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME                                          
   <chr>                                         
 1 NIFTY 100                                     
 2 NIFTY 100 TR                                  
 3 Nifty 1D Rate Index                           
 4 NIFTY 200                                     
 5 NIFTY 200 TR                                  
 6 NIFTY 50                                      
 7 NIFTY 50 ARBITRAGE                            
 8 NIFTY 50 FUTURES INDEX                        
 9 NIFTY 50 FUTURES TR INDEX                     
10 NIFTY 50 TR                                   
11 NIFTY 500                                     
12 NIFTY 500 TR                                  
13 NIFTY ADITYA BIRLA GROUP                      
14 NIFTY ADITYA BIRLA GROUP TR                   
15 NIFTY ALPHA 50                                
16 NIFTY ALPHA 50 TR                

In [9]:
maxDtBse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtBse) %>%
     select(NAME) %>%
     print(n=20)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME                                    
   <chr>                                   
 1 NIFTY 100                               
 2 NIFTY 100 TR                            
 3 Nifty 1D Rate Index                     
 4 NIFTY 200                               
 5 NIFTY 200 TR                            
 6 NIFTY 50                                
 7 NIFTY 50 ARBITRAGE                      
 8 NIFTY 50 FUTURES INDEX                  
 9 NIFTY 50 FUTURES TR INDEX               
10 NIFTY 50 TR                             
11 NIFTY 500                               
12 NIFTY 500 TR                            
13 NIFTY ADITYA BIRLA GROUP                
14 NIFTY ADITYA BIRLA GROUP TR             
15 NIFTY ALPHA 50                          
16 NIFTY ALPHA 50 TR                       
17 NIFTY ALPHA LOW-VOLATILITY 30           
18 NIFTY ALPHA LOW-VOLATILITY 30 TR        
19 NIF

In [10]:
maxDtBse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]

nifty_indices <- indices$NseTimeSeries() %>%
                 group_by(NAME) %>%
                 filter(TIME_STAMP == maxDtBse) %>%
                 select(NAME) %>%
                 print(n=20)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME                                    
   <chr>                                   
 1 NIFTY 100                               
 2 NIFTY 100 TR                            
 3 Nifty 1D Rate Index                     
 4 NIFTY 200                               
 5 NIFTY 200 TR                            
 6 NIFTY 50                                
 7 NIFTY 50 ARBITRAGE                      
 8 NIFTY 50 FUTURES INDEX                  
 9 NIFTY 50 FUTURES TR INDEX               
10 NIFTY 50 TR                             
11 NIFTY 500                               
12 NIFTY 500 TR                            
13 NIFTY ADITYA BIRLA GROUP                
14 NIFTY ADITYA BIRLA GROUP TR             
15 NIFTY ALPHA 50                          
16 NIFTY ALPHA 50 TR                       
17 NIFTY ALPHA LOW-VOLATILITY 30           
18 NIFTY ALPHA LOW-VOLATILITY 30 TR        
19 NIF

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)